In [15]:
import utilities
import models
import torch
import importlib
importlib.reload(utilities)
importlib.reload(models)


<module 'models' from '/home/kaitlin/repos/equiNAS/models.py'>

In [2]:
model = models.TDRegEquiCNN()
train_loader, validation_loader, test_loader = utilities.get_dataloaders("/home/kaitlin/repos")


In [3]:
from torch.nn import Module
from torch import nn


class LeNet5(Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(256, 120)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(120, 84)
        self.relu4 = nn.ReLU()
        self.fc3 = nn.Linear(84, 10)
        self.relu5 = nn.ReLU()
        self.uuid = "null"
        self.gs = [(0,0) for _ in range(6)]
        self.loss_function = torch.nn.CrossEntropyLoss()

    def forward(self, x):
        y = self.conv1(x)
        y = self.relu1(y)
        y = self.pool1(y)
        y = self.conv2(y)
        y = self.relu2(y)
        y = self.pool2(y)
        y = y.view(y.shape[0], -1)
        y = self.fc1(y)
        y = self.relu3(y)
        y = self.fc2(y)
        y = self.relu4(y)
        y = self.fc3(y)
        y = self.relu5(y)
        return y

model = LeNet5() 

In [4]:
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.transforms import RandomRotation, Pad, Resize, ToTensor, Compose
from torchvision.transforms.functional import InterpolationMode
from torchvision import datasets

class MnistDataset(Dataset):
    
    def __init__(self, mode, transform=None, path_to_dir='~'):
        assert mode in ['train', 'test']
            
        if mode == "train":
            file = path_to_dir+"/data/mnist_train.amat"
        else:
            file = path_to_dir+"/data/mnist_test.amat"
        
        self.transform = transform
        print(file)
        data = np.genfromtxt(file, delimiter='   ')
        print(np.shape(data))
            
        self.images = data[:, :-1].reshape(-1, 28, 28).astype(np.float32)
        self.labels = data[:, -1].astype(np.int64)
        self.num_samples = len(self.labels)

        #self.images = np.pad(self.images, pad_width=((0,0), (2, 3), (2, 3)), mode='edge')

    
    def __getitem__(self, index):
        image, label = self.images[index], self.labels[index]
        image = Image.fromarray(image, mode='F')
        if self.transform is not None:
            image = self.transform(image)
        return image, label
    
    def __len__(self):
        return len(self.labels)
    

def get_vanilla_dataloaders(path_to_dir = "~"):
    pad = Pad((0, 0, 1, 1), fill=0)
    resize1 = Resize(87)
    resize2 = Resize(29)
    totensor = ToTensor()
    train_transform = Compose([
        pad,
        resize1,
        RandomRotation(180., interpolation=InterpolationMode.BILINEAR, expand=False),
        resize2,
        totensor,
    ])
    test_transform = Compose([
        pad,
        totensor,
    ])

    mnist_train = MnistDataset(mode='train', transform=test_transform, path_to_dir=path_to_dir)

    shuffle_dataset = True
    random_seed= 42
    validation_split = .2
    dataset_size = mnist_train.num_samples
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))
    if shuffle_dataset :
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)

    #train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=64)
    train_loader = DataLoader(mnist_train, batch_size=64, 
                                            sampler=train_sampler)
    validation_loader = DataLoader(mnist_train, batch_size=64,
                                                    sampler=valid_sampler)

    
    mnist_test = MnistDataset(mode='test', transform=test_transform, path_to_dir=path_to_dir)
    test_loader = DataLoader(mnist_test, batch_size=64)

    return train_loader, validation_loader, test_loader

def get_generic_dataloaders(path_to_dir = "~"):
    pad = Pad((0, 0, 1, 1), fill=0)
    resize1 = Resize(87)
    resize2 = Resize(29)
    totensor = ToTensor()
    train_transform = Compose([
        pad,
        resize1,
        RandomRotation(180., interpolation=InterpolationMode.BILINEAR, expand=False),
        resize2,
        totensor,
    ])
    test_transform = Compose([
        pad,
        totensor,
    ])
    mnist_train = datasets.MNIST(root='/home/kaitlin/repos/data/mnist_data', 
                                        train=True, 
                                        transform=test_transform,
                                        download=True)

    shuffle_dataset = True
    random_seed= 42
    validation_split = .2
    dataset_size = len(mnist_train.targets)
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))
    if shuffle_dataset :
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(mnist_train, batch_size=64, 
                                            sampler=train_sampler)
    validation_loader = DataLoader(mnist_train, batch_size=64,
                                                    sampler=valid_sampler)

    
    mnist_test = datasets.MNIST(root='/home/kaitlin/repos/data/mnist_data', 
                                        train=False, 
                                        transform=test_transform,
                                        download=True)
    test_loader = DataLoader(mnist_test, batch_size=64)

    return train_loader, validation_loader, test_loader

In [10]:
for lr in [1e0, 8e-1, 7e-1, 6e-1, 5e-1, 4e-1, 3e-1, 2e-1, 1e-1]:
    model = models.TDRegEquiCNN()
    train_loader, validation_loader, test_loader = utilities.get_dataloaders("/home/kaitlin/repos")
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    model.optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    dataloaders = {
        "train": train_loader,
        "validation": validation_loader
    }
    history = {}
    epochs = 15
    history[model.uuid] = {'train': {'loss': [], 
                            'accuracy': [], 
                            'batch': [], 
                            'batchloss': []},
                    'trainsteps': [],
                    'validation' : {'loss': [], 
                                    'accuracy': []},
                    'epochsteps': [],
                    'ghistory': []}
    history[model.uuid]["ghistory"].append(model.gs)
    for epoch in range(epochs):
        for phase in ['train', 'validation']:
            batch = []
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            running_count = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)

                loss = model.loss_function(outputs, labels)

                if phase == 'train':
                    model.optimizer.zero_grad()
                    loss.backward()
                    model.optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.detach() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                running_count += inputs.size(0)
                if phase == "train":
                    batch.append(running_count)
                    history[model.uuid][phase]['batchloss'].append(loss.detach().item())
            epoch_loss = running_loss / running_count
            epoch_acc = running_corrects.float() / running_count

            history[model.uuid][phase]['loss'].append(epoch_loss.item())
            history[model.uuid][phase]['accuracy'].append(epoch_acc.item())
            model.score = epoch_acc.item()

    print(lr, epoch_acc.item())


1.0 0.7912499904632568
0.8 0.8062500357627869
0.7 0.7750000357627869
0.6 0.7866666913032532
0.5 0.8070833683013916
0.4 0.8258333802223206
0.3 0.8229166865348816
0.2 0.7883333563804626
0.1 0.8020833730697632


In [49]:
importlib.reload(models)

model = models.SkipEquiCNN()
for inputs, labels in train_loader:
    inputs_ = inputs.clone()
    outputs = model(inputs)
    break

for tochange in range(6):
    print(tochange)
    parent = models.SkipEquiCNN(gs = [(0,2) for _ in range(tochange+1)]+[(0,1) for _ in range(5-tochange)], ordered = True)
    child = parent.offspring(tochange, (0,1))
    print(torch.allclose(parent(inputs_.clone()), child(inputs_.clone()), rtol = 1e-1, atol = 1e-1))
        


0
True
1
True
2
True
3
False
4
True
5


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x3888 and 48x64)